In [2]:
# path to json
DATA_PATH = "data_10.json"

In [3]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

def load_data(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")
    return  X, y

In [4]:
# Ensure you define DATA_PATH before calling the function or replace DATA_PATH with the actual path string.
X, y = load_data(DATA_PATH)

Data succesfully loaded!


In [5]:
def display_data(X, y, num_samples=5):
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}\n")

    print(f"First {num_samples} entries of X:")
    print(X[:num_samples])

    print(f"\nFirst {num_samples} entries of y:")
    print(y[:num_samples])

# Call the display function
display_data(X, y)

Shape of X: (9986, 130, 13)
Shape of y: (9986,)

First 5 entries of X:
[[[ -32.52093887  101.69689941  -12.49266434 ...    0.91859376
      6.12813425    7.43504047]
  [   4.63589716   92.01476288  -14.43618774 ...   -3.12150717
     10.91743088   12.23798275]
  [   5.13217115   93.25161743  -15.10477638 ...   -6.58130455
      6.40962601    9.9592886 ]
  ...
  [ -26.90529823   50.987854      7.74356651 ...    5.58424664
     -4.36807489   -1.8676579 ]
  [ -44.19861984   67.88891602   -1.54656506 ...   -5.91294098
     -1.77724183   -1.53501415]
  [ -82.0346756    90.83355713   -0.68910015 ...   -9.12341118
     -2.63756299   -6.29706573]]

 [[-118.24279022  105.50691223    2.0165503  ...   -6.59367657
     -5.19090176  -20.08145142]
  [ -95.36672211   90.11253357   -5.41918373 ...  -13.54324532
     -6.37954044  -13.70101547]
  [ -82.44755554   74.75208282   -3.18817306 ...  -18.20688629
     -9.79681206   -9.82308388]
  ...
  [  26.10294342   11.1639576    12.10066891 ...   10.065635

In [6]:
# Check the unique labels to determine the number of classes
num_classes = len(np.unique(y))
print(num_classes)

10


In [7]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, y_train.shape)

(6990, 130, 13) (6990,)


In [8]:
print("Unique labels:", np.unique(y_train))

Unique labels: [ 0  1  2  3  5  6  7  8  9 10]


In [9]:
def transform_labels(y):
    # Create a dictionary to map original labels to new labels
    unique_labels = np.unique(y)
    label_mapping = {original_label: idx for idx, original_label in enumerate(unique_labels)}

    # Use the map to transform y
    return np.array([label_mapping[label] for label in y])

y = transform_labels(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
print("Unique labels:", np.unique(y_train))
print("Min label:", y_train.min())
print("Max label:", y_train.max())

Unique labels: [0 1 2 3 4 5 6 7 8 9]
Min label: 0
Max label: 9


In [11]:
class_names = ["Blues", "Classical", "Country", "Disco", "Hip-Hop", "Jazz", "Metal", "Pop", "Reggae", "Rock"]

# Based on the unique labels you provided: [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
# Let's create a direct mapping from these labels to the class names:

label_to_class_mapping = {idx: class_name for idx, class_name in enumerate(class_names)}

In [12]:
print("Label to class mapping:")
for label, class_name in label_to_class_mapping.items():
    print(f"Label {label}: {class_name}")

Label to class mapping:
Label 0: Blues
Label 1: Classical
Label 2: Country
Label 3: Disco
Label 4: Hip-Hop
Label 5: Jazz
Label 6: Metal
Label 7: Pop
Label 8: Reggae
Label 9: Rock


In [13]:
print("Min label:", y_train.min())
print("Max label:", y_train.max())

Min label: 0
Max label: 9


In [14]:
import cv2

def preprocess_data(X):
    # Resize each image in the dataset and duplicate its channel to have 3 channels
    X_processed = np.array([cv2.resize(x, (128, 128)) for x in X])
    X_processed = np.stack((X_processed,)*3, axis=-1)
    return X_processed

X_train_processed = preprocess_data(X_train)
X_test_processed = preprocess_data(X_test)

In [15]:
print("X_train contains NaN:", np.isnan(X_train_processed).any())
print("X_test contains NaN:", np.isnan(X_test_processed).any())
print("y_train contains NaN:", np.isnan(y_train).any())
print("y_test contains NaN:", np.isnan(y_test).any())

X_train contains NaN: False
X_test contains NaN: False
y_train contains NaN: False
y_test contains NaN: False


In [16]:
!nvidia-smi

Mon Nov 27 01:52:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8              12W / 130W |     16MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential

# Load the pre-trained ResNet50 model without the top layer
# Adjust the 'input_shape' to match the shape of your spectrograms
input_shape = (128, 128, 3)  # Example shape, adjust based on your data preprocessing
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
base_model.trainable = False  # Freeze the base model

In [23]:
# Create a new model on top
model_resnet = Sequential([
    base_model,
    GlobalAveragePooling2D(),

    # Dense layers from model_9
    # ... (add your model_9 layers here)
    # You can keep using Conv and MaxPooling layers or skip straight to Flatten
    # Depending on your dataset, you may experiment with adding additional Conv layers or start flattening
    Flatten(),
    # Dense layers from model_9
    Dense(256, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])



In [25]:
# Optimizer, compile model, callbacks
optimiser = Adam(learning_rate=0.0005)
model_resnet.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_resnet50_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [27]:
# Now you can fit the model with the processed data
history = model_resnet.fit(X_train_processed, y_train, 
                             validation_data=(X_test_processed, y_test), 
                             batch_size=32, epochs=100, 
                             callbacks=[lr_scheduler, early_stopping, checkpoint])

Epoch 1/100
219/219 [==============================] - ETA: 0s - loss: 2.4595 - accuracy: 0.3990
Epoch 1: val_loss improved from inf to 1.95384, saving model to best_resnet50_model.h5


D:\Anaconda\envs\tf_gpu_env\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


219/219 [==============================] - 155s 694ms/step - loss: 2.4595 - accuracy: 0.3990 - val_loss: 1.9538 - val_accuracy: 0.5347 - lr: 5.0000e-04
Epoch 2/100
219/219 [==============================] - ETA: 0s - loss: 2.0526 - accuracy: 0.4856
Epoch 2: val_loss improved from 1.95384 to 1.83336, saving model to best_resnet50_model.h5
219/219 [==============================] - 144s 657ms/step - loss: 2.0526 - accuracy: 0.4856 - val_loss: 1.8334 - val_accuracy: 0.5494 - lr: 5.0000e-04
Epoch 3/100
219/219 [==============================] - ETA: 0s - loss: 1.8465 - accuracy: 0.5475
Epoch 3: val_loss improved from 1.83336 to 1.65746, saving model to best_resnet50_model.h5
219/219 [==============================] - 136s 622ms/step - loss: 1.8465 - accuracy: 0.5475 - val_loss: 1.6575 - val_accuracy: 0.6011 - lr: 5.0000e-04
Epoch 4/100
219/219 [==============================] - ETA: 0s - loss: 1.7077 - accuracy: 0.5761
Epoch 4: val_loss improved from 1.65746 to 1.62877, saving model to bes

In [22]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, LeakyReLU, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential

# Load the pre-trained DenseNet201 model without the top layer
input_shape = (224, 224, 3)  # Example shape, adjust based on your data preprocessing
base_model = DenseNet201(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
base_model.trainable = False  # Freeze the base model

In [23]:
# Create a new model on top
model_densenet = Sequential([
    base_model,
    GlobalAveragePooling2D(),

    # Dense layers
    Dense(256, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    # Output layer
    Dense(num_classes, activation='softmax')
])

In [24]:
# Optimizer, compile model, callbacks
optimiser = Adam(learning_rate=0.0005)
model_densenet.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_densenet201_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [26]:
# Now you can fit the model with the processed data
history = model_densenet.fit(X_train_processed, y_train, 
                             validation_data=(X_test_processed, y_test), 
                             batch_size=32, epochs=100, 
                             callbacks=[lr_scheduler, early_stopping, checkpoint])

Epoch 1/100
 56/219 [======>.......................] - ETA: 5:34 - loss: 2.8411 - accuracy: 0.2796

KeyboardInterrupt: 

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# Load the pre-trained DenseNet121 model without the top layer
# Adjust the 'input_shape' to match the shape of your spectrograms
input_shape = (128, 128, 3)  # Example shape, adjust based on your data preprocessing
base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
base_model.trainable = False  # Freeze the base model

# Create a new model on top
model_densenet = Sequential([
    base_model,
    GlobalAveragePooling2D(),

    # Layers from model_9
    # Add your model_9 layers here
    # You can keep using Conv and MaxPooling layers or skip straight to Flatten
    Flatten(),
    Dense(256, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(),
    BatchNormalization(),
    Dropout(0.3),

    # Output layer
    Dense(num_classes, activation='softmax')
])

In [18]:
# Optimizer, compile model, and define callbacks
optimiser = Adam(learning_rate=0.0005)
model_densenet.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_densenet_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [19]:
# Replace 'X_train_processed' and 'y_train' with your training data and labels
# Replace 'X_test_processed' and 'y_test' with your validation/test data and labels
# Make sure to preprocess your data before using it
history = model_densenet.fit(X_train_processed, y_train, 
                             validation_data=(X_test_processed, y_test), 
                             batch_size=32, epochs=100, 
                             callbacks=[lr_scheduler, early_stopping, checkpoint])

Epoch 1/100
219/219 [==============================] - ETA: 0s - loss: 2.4063 - accuracy: 0.3619
Epoch 1: val_loss improved from inf to 1.98203, saving model to best_densenet_model.h5


D:\Anaconda\envs\tf_gpu_env\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


219/219 [==============================] - 175s 780ms/step - loss: 2.4063 - accuracy: 0.3619 - val_loss: 1.9820 - val_accuracy: 0.4499 - lr: 5.0000e-04
Epoch 2/100
219/219 [==============================] - ETA: 0s - loss: 2.0141 - accuracy: 0.4475
Epoch 2: val_loss improved from 1.98203 to 1.84227, saving model to best_densenet_model.h5
219/219 [==============================] - 225s 1s/step - loss: 2.0141 - accuracy: 0.4475 - val_loss: 1.8423 - val_accuracy: 0.4710 - lr: 5.0000e-04
Epoch 3/100
219/219 [==============================] - ETA: 0s - loss: 1.8960 - accuracy: 0.4805
Epoch 3: val_loss improved from 1.84227 to 1.71029, saving model to best_densenet_model.h5
219/219 [==============================] - 213s 973ms/step - loss: 1.8960 - accuracy: 0.4805 - val_loss: 1.7103 - val_accuracy: 0.5150 - lr: 5.0000e-04
Epoch 4/100
219/219 [==============================] - ETA: 0s - loss: 1.8242 - accuracy: 0.4983
Epoch 4: val_loss improved from 1.71029 to 1.61585, saving model to best_d

219/219 [==============================] - ETA: 0s - loss: 1.2010 - accuracy: 0.6521
Epoch 27: val_loss improved from 1.23953 to 1.16422, saving model to best_densenet_model.h5
219/219 [==============================] - 161s 735ms/step - loss: 1.2010 - accuracy: 0.6521 - val_loss: 1.1642 - val_accuracy: 0.6609 - lr: 2.5000e-04
Epoch 28/100
219/219 [==============================] - ETA: 0s - loss: 1.1870 - accuracy: 0.6515
Epoch 28: val_loss did not improve from 1.16422
219/219 [==============================] - 189s 863ms/step - loss: 1.1870 - accuracy: 0.6515 - val_loss: 1.1901 - val_accuracy: 0.6539 - lr: 2.5000e-04
Epoch 29/100
219/219 [==============================] - ETA: 0s - loss: 1.1746 - accuracy: 0.6639
Epoch 29: val_loss did not improve from 1.16422
219/219 [==============================] - 196s 898ms/step - loss: 1.1746 - accuracy: 0.6639 - val_loss: 1.1800 - val_accuracy: 0.6502 - lr: 2.5000e-04
Epoch 30/100
219/219 [==============================] - ETA: 0s - loss: 1.1

Epoch 53/100
219/219 [==============================] - ETA: 0s - loss: 0.9640 - accuracy: 0.7293
Epoch 53: val_loss improved from 1.01722 to 1.00853, saving model to best_densenet_model.h5
219/219 [==============================] - 140s 640ms/step - loss: 0.9640 - accuracy: 0.7293 - val_loss: 1.0085 - val_accuracy: 0.7126 - lr: 6.2500e-05
Epoch 54/100
219/219 [==============================] - ETA: 0s - loss: 0.9705 - accuracy: 0.7236
Epoch 54: val_loss did not improve from 1.00853
219/219 [==============================] - 144s 656ms/step - loss: 0.9705 - accuracy: 0.7236 - val_loss: 1.0234 - val_accuracy: 0.6939 - lr: 6.2500e-05
Epoch 55/100
219/219 [==============================] - ETA: 0s - loss: 0.9777 - accuracy: 0.7189
Epoch 55: val_loss did not improve from 1.00853
219/219 [==============================] - 148s 675ms/step - loss: 0.9777 - accuracy: 0.7189 - val_loss: 1.0166 - val_accuracy: 0.6996 - lr: 6.2500e-05
Epoch 56/100
219/219 [==============================] - ETA: 0

219/219 [==============================] - 154s 705ms/step - loss: 0.9184 - accuracy: 0.7431 - val_loss: 0.9755 - val_accuracy: 0.7083 - lr: 3.1250e-05
Epoch 80/100
219/219 [==============================] - ETA: 0s - loss: 0.9105 - accuracy: 0.7409
Epoch 80: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 80: val_loss did not improve from 0.96784
219/219 [==============================] - 151s 692ms/step - loss: 0.9105 - accuracy: 0.7409 - val_loss: 0.9718 - val_accuracy: 0.7106 - lr: 3.1250e-05
Epoch 81/100
219/219 [==============================] - ETA: 0s - loss: 0.9000 - accuracy: 0.7446
Epoch 81: val_loss improved from 0.96784 to 0.96730, saving model to best_densenet_model.h5
219/219 [==============================] - 154s 705ms/step - loss: 0.9000 - accuracy: 0.7446 - val_loss: 0.9673 - val_accuracy: 0.7130 - lr: 1.5625e-05
Epoch 82/100
219/219 [==============================] - ETA: 0s - loss: 0.8969 - accuracy: 0.7382
Epoch 82: val_loss improved fro